In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
import os

# Define directories
processed_data_dir = r'C:\Users\ACER\gitClones\QuickDrawGame\model_training\data\processed_data'

# Load image and label files
def load_data():
    images = []
    labels = []
    
    # Loop through all categories (replace with your actual labels)
    labels_list = ['airplane', 'dog', 'cat', 'fish', 'truck', 'car', 'rabbit', 'elephant']
    
    for label in labels_list:
        # Load the corresponding images and labels for each batch
        for batch_index in range(1, 11):  # Assuming 10 batches per label
            # Correct file paths for images and one-hot labels
            image_file = os.path.join(processed_data_dir, f'full_raw_{label}_images_{batch_index}.npy')
            label_file = os.path.join(processed_data_dir, f'full_raw_{label}_one_hot_labels_{batch_index}.npy')
            
            # Load images and one-hot encoded labels
            images_batch = np.load(image_file)
            labels_batch = np.load(label_file)
            
            images.append(images_batch)
            labels.append(labels_batch)
    
    # Concatenate all batches of images and labels
    images = np.concatenate(images, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    # Reshape the images to ensure they're in the correct format (e.g., 28x28x1)
    images = images.reshape(-1, 28, 28, 1)  # Assuming 28x28 grayscale images
    
    return images, labels

# Load the data
images, labels = load_data()

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")


Training data shape: (6400, 28, 28, 1)
Validation data shape: (1600, 28, 28, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [14]:
from tensorflow.keras.backend import clear_session

clear_session()

In [15]:
# Load the MobileNetV2 model pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model to keep the pre-trained weights
base_model.trainable = False

# Create the model
model = Sequential([
    # Add the pre-trained base model
    base_model,
    
    # Global average pooling layer to reduce dimensionality
    GlobalAveragePooling2D(),
    
    # Add a dense layer for custom output
    Dense(128, activation='relu'),
    
    # Dropout layer to avoid overfitting
    Dropout(0.5),
    
    # Output layer with 19 classes (your number of categories)
    Dense(19, activation='softmax')  # 19 categories, softmax for multi-class classification
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

C:\Users\ACER\AppData\Local\Temp\ipykernel_21052\364207943.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 1, 1, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 19)             │         2,451 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,424,403 (9.25 MB)

 Trainable params: 166,419 (650.07 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [16]:
import cv2
import numpy as np

# Resize images to 32x32 and convert grayscale to RGB (3 channels)
def preprocess_images(images):
    images_resized = []
    for img in images:
        img_resized = cv2.resize(img, (32, 32))  # Resize to (32, 32)
        
        # If the image is grayscale (1 channel), convert it to RGB (3 channels)
        if img_resized.ndim == 2:  # If the image is grayscale (only 2 dimensions: height and width)
            img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)  # Convert to RGB
        
        # If the image has only 1 channel (grayscale), convert it to RGB
        elif img_resized.shape[-1] == 1:  # Check if the image has only 1 channel (grayscale)
            img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)  # Convert to RGB
        
        images_resized.append(img_resized)
    
    return np.array(images_resized)

# Assuming X_train and X_val are your training and validation image datasets
X_train = preprocess_images(X_train)
X_val = preprocess_images(X_val)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")


Training data shape: (6400, 32, 32, 3)
Validation data shape: (1600, 32, 32, 3)


In [17]:
# Train the model with your training and validation data
history = model.fit(
    X_train, 
    y_train, 
    epochs=10,  # Adjust epochs based on your needs
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=2
)

# Save the trained model
model.save('quick_draw_model.h5')

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=2)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/10
200/200 - 12s - 60ms/step - accuracy: 0.1244 - loss: 2.3988 - val_accuracy: 0.1138 - val_loss: 2.1155
Epoch 2/10
200/200 - 5s - 27ms/step - accuracy: 0.1247 - loss: 2.1188 - val_accuracy: 0.1194 - val_loss: 2.0891
Epoch 3/10
200/200 - 5s - 27ms/step - accuracy: 0.1250 - loss: 2.1033 - val_accuracy: 0.1138 - val_loss: 2.0853
Epoch 4/10
200/200 - 5s - 27ms/step - accuracy: 0.1230 - loss: 2.0983 - val_accuracy: 0.1231 - val_loss: 2.0847
Epoch 5/10
200/200 - 6s - 28ms/step - accuracy: 0.1216 - loss: 2.0945 - val_accuracy: 0.1250 - val_loss: 2.0826
Epoch 6/10
200/200 - 6s - 30ms/step - accuracy: 0.1269 - loss: 2.0898 - val_accuracy: 0.1138 - val_loss: 2.0849
Epoch 7/10
200/200 - 6s - 30ms/step - accuracy: 0.1247 - loss: 2.0870 - val_accuracy: 0.1138 - val_loss: 2.0844
Epoch 8/10
200/200 - 6s - 30ms/step - accuracy: 0.1184 - loss: 2.0876 - val_accuracy: 0.1138 - val_loss: 2.0828
Epoch 9/10
200/200 - 6s - 31ms/step - accuracy: 0.1189 - loss: 2.0891 - val_accuracy: 0.1194 - val_loss

50/50 - 1s - 25ms/step - accuracy: 0.1231 - loss: 2.0832
Validation Loss: 2.083224296569824
Validation Accuracy: 0.12312500178813934


In [18]:
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
history_finetune = model.fit(
    X_train, 
    y_train, 
    epochs=5,  # Fine-tune for fewer epochs
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=2
)

# Save the fine-tuned model
model.save('quick_draw_finetuned_model.h5')


Epoch 1/5
200/200 - 12s - 58ms/step - accuracy: 0.1275 - loss: 2.0842 - val_accuracy: 0.1219 - val_loss: 2.0831
Epoch 2/5
200/200 - 6s - 28ms/step - accuracy: 0.1314 - loss: 2.0838 - val_accuracy: 0.1219 - val_loss: 2.0830
Epoch 3/5
200/200 - 6s - 29ms/step - accuracy: 0.1211 - loss: 2.0853 - val_accuracy: 0.1219 - val_loss: 2.0829
Epoch 4/5
200/200 - 6s - 29ms/step - accuracy: 0.1209 - loss: 2.0842 - val_accuracy: 0.1219 - val_loss: 2.0828
Epoch 5/5
200/200 - 6s - 30ms/step - accuracy: 0.1266 - loss: 2.0833 - val_accuracy: 0.1231 - val_loss: 2.0827
